In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-seq/train (1)/kaggle/working/train/state.json
/kaggle/input/dataset-seq/train (1)/kaggle/working/train/dataset_info.json
/kaggle/input/dataset-seq/train (1)/kaggle/working/train/dataset.arrow
/kaggle/input/dataset-seq/test (1)/kaggle/working/test/state.json
/kaggle/input/dataset-seq/test (1)/kaggle/working/test/dataset_info.json
/kaggle/input/dataset-seq/test (1)/kaggle/working/test/dataset.arrow
/kaggle/input/dataset-seq/eval (1)/kaggle/working/eval/state.json
/kaggle/input/dataset-seq/eval (1)/kaggle/working/eval/dataset_info.json
/kaggle/input/dataset-seq/eval (1)/kaggle/working/eval/dataset.arrow


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import PreTrainedModel
from transformers import pipeline, set_seed, TextGenerationPipeline
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.tokenization_utils_base import BatchEncoding
from transformers import DataCollatorForLanguageModeling, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import set_seed
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from datasets.iterable_dataset import IterableDataset
from torch import nn
from torch import Tensor
from typing import List, Dict, Any
import pandas as pd
import re
import json
import multiprocessing
import torch

model_id: str = 't5-base' # you can also try: distilgpt2 gpt2
num_train_epochs: int = 1
instruction_format: str = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n"
    "\n"
    "### Question:\n"
    "{question}"
    "\n\n"
    "### Answer:\n"
    "{answer}"
)
device: torch.device = torch.device("cuda") \
  if torch.cuda.is_available() else torch.device("cpu")
device

2024-05-08 05:34:51.572113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 05:34:51.572211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 05:34:51.686378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


device(type='cuda')

In [3]:
test_dataset: Dataset = load_from_disk('/kaggle/input/dataset-seq/test (1)/kaggle/working/test')
train_dataset: Dataset = load_from_disk('/kaggle/input/dataset-seq/train (1)/kaggle/working/train')
eval_dataset: Dataset = load_from_disk('/kaggle/input/dataset-seq/eval (1)/kaggle/working/eval')

train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 198000
})

In [4]:
tokenizer: PreTrainedTokenizerBase = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model: nn.Module = AutoModelForSeq2SeqLM.from_pretrained(model_id)
print(f"model: {model}")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

model: T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropou

In [5]:
question_list = []
answer_list = []
for x in test_dataset:
    question_list.append(tokenizer.decode(x['input_ids'], skip_special_tokens=True))
    answer_list.append(tokenizer.decode(x['labels'], skip_special_tokens=True))
print(len(question_list))
print(len(answer_list))
print(question_list[0])
print(answer_list[0])

1000
1000
safari download button render shift leav rest page load might comment box
Hey Mason, wed like to get a look at this. Would you mind raising a support ticket here and including a screencast of the behavior youre experiencing? To create a screencast recording your issue, use QuickTime Player 10.x. You can download this... 1/2


In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [7]:
print('for seq2seq')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_id}-stackexchange",
    per_device_train_batch_size=4, # important, to prevent GPU OOM
    num_train_epochs=3,
    push_to_hub=False,
    save_strategy='no',
    learning_rate=2e-4,
    adam_beta1=0.8,
    logging_steps=2000,
)
# for more args, visit:
# https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


for seq2seq


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
2000,1.653000
4000,1.511500
6000,1.469100
8000,1.422800
10000,1.392400
12000,1.353900
14000,1.329600
16000,1.304700
18000,1.257700
20000,1.261300


TrainOutput(global_step=148500, training_loss=0.9834188559175742, metrics={'train_runtime': 20056.1619, 'train_samples_per_second': 29.617, 'train_steps_per_second': 7.404, 'total_flos': 4.521512337408e+16, 'train_loss': 0.9834188559175742, 'epoch': 3.0})

In [8]:
trainer.save_model('t5-base')

In [9]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2e42d230818701d77b5ae6ee589fff5f06f366c9abc52fdc110a4071ce9964ba
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [10]:
import evaluate
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')

In [11]:
set_seed(87)
def generate_text_sampling_top_p_nucleus_22(
    input_prompt: str,
    min_length: str = 3,
    max_length: int = 64,
    top_p: float = 0.22,
  ) -> str:
  encoded_input: BatchEncoding = tokenizer(input_prompt, return_tensors='pt').to(device)
  sampling_output_tensor: Tensor = model.generate(
      **encoded_input,
      min_length=min_length,
      max_length=max_length,
      pad_token_id=50256,
      do_sample=True,
      top_p=top_p,
      top_k=0,
  )
  sampling_output_text: str = tokenizer.decode(sampling_output_tensor[0], skip_special_tokens=True)
  return sampling_output_text

In [12]:
def evaluate_bleu_rouge(generate_type):
    predictions = []
    for q in question_list:
        predictions.append(generate_type(q))
    results_bleu = bleu.compute(predictions=predictions, references=answer_list)
    results_rouge = rouge.compute(predictions=predictions, references=answer_list)
    return (results_bleu, results_rouge)

In [13]:
for i in range(49, 59):
    print(f'question: {question_list[i]}')
    print(f'expected answer: {answer_list[i]}')
    print(f'answer: {generate_text_sampling_top_p_nucleus_22(input_prompt=question_list[i])}')
    print('+++++++++++++++++++++++++')


question: point onli see 100 000 posit see 2000 neg eye onli care posit neg experi
expected answer: We care deeply about both and understand the effect a negative experience can have. As mentioned earlier, were always here, so please let us know if there's been a problem with a recent order or anything else we can look at with you!
answer: We care deeply about your post as well as the effect a negative experience can have. At this time, the 2000+ positions are still under review. We're always working to improve our services and look forward to seeing you again soon.
+++++++++++++++++++++++++
question: hi travel companion might leav hi wallet flight thi afternoon file report hope someon grab befor leav friday ani help
expected answer: I'm sorry to hear that, Candace! Our search and rescue Team will use the information you provided in the report to match the wallet against anything turned into us, and reach out to him with any information. I hope we're able to reunite your friend with hi

In [14]:
print('1000 data')

print(evaluate_bleu_rouge(generate_text_sampling_top_p_nucleus_22))

1000 data
({'bleu': 0.13823181904613985, 'precisions': [0.44407025693133984, 0.21052474211391836, 0.1642780089382031, 0.14422006678327237], 'brevity_penalty': 0.6371886553528499, 'length_ratio': 0.689327382977446, 'translation_length': 34445, 'reference_length': 49969}, {'rouge1': 0.33273831443983615, 'rouge2': 0.15504185627456313, 'rougeL': 0.26882082909449956, 'rougeLsum': 0.2687193438727938})
